In [2]:
import pandas as pd

dataDf=pd.read_csv("pnnl_datasets.csv")
dataDf.head()

,Organism,Sample,Dataset,Instrument,Separation Type,Type,Acquisition Start,Acquisition End,Scan Count,Acq Length
0,Acidiphilium_cryptum_JF-5,Acryp-01,Acryp-01a_1Dec09_Sphinx_09-11-27,LTQ_2,LC-ISCO-Formic_100minute,MS-MSn,12/2/2009 5:02 AM,12/2/2009 6:42 AM,17919,100
1,Acidiphilium_cryptum_JF-5,Acryp-01,Acryp-01b_1Dec09_Sphinx_09-11-27,LTQ_2,LC-ISCO-Formic_100minute,MS-MSn,12/2/2009 1:34 PM,12/2/2009 3:14 PM,17824,100
2,Acidiphilium_cryptum_JF-5,Acryp-01,Acryp-01c_25Nov09_Sphinx_09-11-26,LTQ_2,LC-ISCO-Formic_100minute,MS-MSn,11/28/2009 1:33 AM,11/28/2009 3:13 AM,16997,100
3,Acidiphilium_cryptum_JF-5,Acryp-02,Acryp-02a_1Dec09_Sphinx_09-11-27,LTQ_2,LC-ISCO-Formic_100minute,MS-MSn,12/1/2009 3:35 PM,12/1/2009 5:15 PM,18141,100
4,Acidiphilium_cryptum_JF-5,Acryp-02,Acryp-02b_1Dec09_Sphinx_09-11-27,LTQ_2,LC-ISCO-Formic_100minute,MS-MSn,12/3/2009 4:48 AM,12/3/2009 6:28 AM,17545,100


In [3]:
lookupDf=pd.read_csv("pnnl_lookup.csv")
lookupDf.head()

,scientific_name,tax_id,dir_name,genome_flag
0,Acidiphilium cryptum JF-5,349163.0,Acidiphilium_cryptum_JF-5,0
1,Actinosynnema mirum DSM 43827,446462.0,Actinosynnema_mirum_DSM_43827,1
2,Anabaena variabilis,1172.0,Anabaena_variabilis,0
3,Anaeromyxobacter dehalogenans,161493.0,Anaeromyxobacter_dehalogenans,0
4,Anaplasma phagocytophilum,948.0,Anaplasma_phagocytophilium,0


* (down)genome_flagが立っているものに関しては、dir_nameはOrganismと対応していそう.

In [5]:
organism_set=set(dataDf["Organism"])

for dirName in (lookupDf[lookupDf["genome_flag"]==1])["dir_name"]:
    if not(dirName in organism_set):
        print(dirName)

* genome_flagが立っている種について、情報をまとめる

In [7]:
dct_lst=[]
for dirName in (lookupDf[lookupDf["genome_flag"]==1])["dir_name"]:
    dct={}
    dct["dir_name"]=dirName
    dct["total"]=len(dataDf[(dataDf["Organism"]==dirName)])
    dct["HMS"]=len(dataDf[(dataDf["Organism"]==dirName) & (dataDf["Type"]!="MS-MSn")])
    dct_lst.append(dct)

In [8]:
sumDf=pd.DataFrame(dct_lst)#sum for summarize
sumDf=sumDf[["dir_name", "total", "HMS"]]
sumDf[sumDf["HMS"]>0]

,dir_name,total,HMS
1,Arthrobacter_sp_FB24,353,36
2,Bartonella_henselae_Houston-1,160,160
5,Cyanobacterium_synechocystis_PCC6803,279,174
6,Cyanothece_sp_ATCC51142,1404,569
7,Cyanothece_strain_PCC7424,109,83
8,Cyanothece_strain_PCC7425,104,98
9,Cyanothece_strain_PCC7822,166,160
10,Cyanothece_strain_PCC8801,90,90
11,Dehalococcoides_ethenogenes,437,430
13,Enterobacter_cloacae_SCF1,58,58


* HMSが存在する25種に対して最大で30ファイル（暫定）を抽出する

In [10]:
thres=30

dct_lst=[]
for dirName in (lookupDf[lookupDf["genome_flag"]==1])["dir_name"]:
    filteredDf=dataDf[(dataDf["Organism"]==dirName) & (dataDf["Type"]!="MS-MSn")]
    if(filteredDf.shape[0]>0):
        sampledDf=filteredDf.sample(min(thres,filteredDf.shape[0]))
        for _i, row in sampledDf.iterrows():
            dct_lst.append(dict(row))

In [11]:
outDf=pd.DataFrame(dct_lst)
outDf=outDf[dataDf.columns]
outDf.head()

,Organism,Sample,Dataset,Instrument,Separation Type,Type,Acquisition Start,Acquisition End,Scan Count,Acq Length
0,Arthrobacter_sp_FB24,Arthro_009,Arthro_009-LTQFT3_04Oct05_Andro_0705-06,LTQ_FT1,LC-ISCO-Standard,HMS-MSn,10/4/2005 5:10 PM,10/4/2005 6:50 PM,14723,100
1,Arthrobacter_sp_FB24,Arthro_002,Arthro_002-LTQFT2_29Sep05_Andro_0705-06,LTQ_FT1,LC-ISCO-Standard,HMS-MSn,9/29/2005 2:58 AM,9/29/2005 4:38 AM,11808,100
2,Arthrobacter_sp_FB24,Arthro_009,Arthro_009-LTQFT1_04Oct05_Andro_0705-06,LTQ_FT1,LC-ISCO-Standard,HMS-MSn,10/4/2005 12:21 PM,10/4/2005 2:01 PM,14135,100
3,Arthrobacter_sp_FB24,Arthro_010,Arthro_010-LTQFT2_04Oct05_Andro_0705-06,LTQ_FT1,LC-ISCO-Standard,HMS-MSn,10/4/2005 9:59 PM,10/4/2005 11:39 PM,14031,100
4,Arthrobacter_sp_FB24,Arthro_002,Arthro_002-LTQFT3_29Sep05_Andro_0705-06,LTQ_FT1,LC-ISCO-Standard,HMS-MSn,9/29/2005 8:03 AM,9/29/2005 9:43 AM,12582,100


In [12]:
outDf.to_csv("sampled_pnnl_datasets.csv",index=False)